# openCypher Workshop - Section 01: Select & Filter Query Steps
In this section, you will start with some basic openCypher queries that are used to find vertices or edges with given attributes.  Those attributes can be labels, unique IDs, or properties.  Feel free to change any of the queries in this (and subsequent) notebooks to see how changing the query may result in different outcomes.

Happy Traversing!

## Graph Model

We will be using a modified version of the IMDb (Internet Movie Database) data set for this workshop.  We have converted this data into the following graph data model representing certain aspects of the data as vertices, edges, and properties.  While not part of this workshop, it is important to understand graph data modeling.  Please refer to our Graph Reference Architectures GitHub repo for more details on how to perform graph data modeling:  https://github.com/aws-samples/aws-dbs-refarch-graph

Here's the application graph data model:

<img src="https://ee-assets-prod-us-east-1.s3.amazonaws.com/modules/f3f89ef4607743429fb01ae23d983197/v1/workshop/images/datamodel-updated.png"/>



## A note on basic terminology

There are a few commonly used terms when working with graph databases. The three main ones being node or vertex, edge or relationship, and property. Different communites tend to have some subset of these they use more often. The Apache TinkerPop Gremlin documentation for example uses the terms Vertex and Edge exclusively whereas users of openCypher more commonly use Node and Relationship. The examples presented in this notebook follow those same conventions.  One other area where the terminology differs is that edge labels in Gremlin are commonly referred to as a type of relationship in openCypher.

## Select a Node and its Properties

When selecting graph elements, bear the following in mind:

 - Neptune allows you to supply custom node and relationship IDs. Use these if possible to lookup a node or relationship by ID. User supplied IDs are not typically used within other databases that support Cypher/openCypher.  However, Neptune does support them.  These can be used in a performant manner to lookup an entity with the closest thing to an O(1) access pattern.
 - If you can't use an ID to lookup an element, use the _minimum number_ of label/type and properties necessary to uniquely identify the element or the set of elements you want to find.
 - When returning results, select only the properties necessary to satisfy your query, rather than returning nodes and relationships in their entirety.

### 01.01 - Find a Node or Relationship by ID

To start, you will perform the most simple of openCypher operations to find a node and a relationship by it's unique ID.  Each node and relationship in Neptune has a unique ID.  IDs can be random (such as UUIDs) or can also be something meaningful, if also unique.  Examples of this can be email addresses signifying a person node.  In the IMDb dataset, IDs are already provided for both Artists and their works (e.g. movies, TV shows, etc.).  For example, if you go to the [IMDb page for Kevin Bacon](https://www.imdb.com/name/nm0000102/) you will notice the last part of the URL is `nm0000102` is the unique identifer for Kevin.

In the following example, we use Kevin's identifer to find his related node.

In [ ]:
%%oc

MATCH (n)
WHERE id(n) = 'nm0000102'
RETURN n

You can do the same with a relationship.  Each relationship has a unique ID.  The original IMDb dataset did not have a first-class representation of the relationships.  We had to manufacture these using the unique IDs of the Artists and their productions.  In this case, we used a composite ID combining the unique IDs of the nodes on either side of a relationship to derive the unique ID for each relationship.  For example, the relationship from [Kevin Bacon](https://www.imdb.com/name/nm0000102/) to [Footloose](https://www.imdb.com/title/tt0087277/) is `tt0087277-actor-nm0000102`.  Try querying for this relationship:

In [ ]:
%%oc

MATCH ()-[e]->()
WHERE id(e) = 'tt0087277-actor-nm0000102'
RETURN e

### 01.02 - Returning Specific Property Values

The default nature of openCypher is to return all associated information for a given node or relationship.  While this is useful during development, it can add overhead if returning more information than what is required for a given application.  Neptune's performance is based heavily on the number of atomic objects that need to be "touched" or returned with a result.  Selecting and return specific properties can help improve the performance of queries.
 
In the following example, let's find the node associated with Kevin Bacon and only return the node's `name` and `birthYear` properties:

In [ ]:
%%oc

MATCH (n)
WHERE id(n) = 'nm0000102'
RETURN n.name, n.birthYear

Here's another example with the movie "Footloose".  Let's return the `title` and `year` properties:

In [ ]:
%%oc

MATCH (n)
WHERE id(n) = 'tt0087277'
RETURN n.title, n.year

### 01.03 - Filtering by Certain Properties

Sometimes you won't know the exact ID for a specific node or relationship.  It is more likely that you would know the value of a given property associated with that node or relationship.  Like SQL, openCypher allows us to supply certain conditions in the `WHERE` clause to look up nodes and relationships by a given property.  Note that the values provided are exact-match and case-sensitive if looking up by string values.
 
In the below example, we want to find all nodes representing the Footloose movie.

In [ ]:
%%oc

MATCH (n)
WHERE n.title = 'Footloose'
RETURN n

We can also use curly bracket notation to define our property filter, so that we don't need to specify it in the `WHERE` clause.

In [ ]:
%%oc

MATCH (n {title:'Footloose'})
RETURN n

With our data set, you'll find that there are two movies called "Footloose." If we had included other media types in our data set, you would have found additional instances of "Footloose" in the form of TV episodes. If we wanted to filter our results to "Footloose" media of a certain type, we could filter on the node label. 

Each relationship and node in Neptune has an associated label. The label denotes the type of data being represented by a set of nodes or relationships. Labels tend to be of very low cardinality. In the case of the IMDb dataset, we have node labels representing artists, movies, genres, people, and places.

openCypher supports labels through two mechanisms. You can supply the label as part of the `MATCH` clause (like below), or you can use either the `labels()` or `type()` functions within the `WHERE` clause. The `labels()` function will return a list containing the labels for a node, and the `type()` function returns a relationship type. This is because in openCypher, labels on relationships are typically referred to as a `type` instead of a label, whereas for vertices they are referred to as a label.

Supplying the label through the `MATCH` clause...

In [ ]:
%%oc

MATCH (n:movie)
WHERE n.title = 'Footloose'
RETURN n

Or you can supply the label through the `WHERE` clause...

In [ ]:
%%oc

MATCH (n)
WHERE n.title = 'Footloose' AND 'movie' IN labels(n)
RETURN n

You should see output representing the two renditiions of the movie "Footloose".  One from 1984 and the more recent from 2011.

### 01.04 - Using Conditionals

Beyond property filters, there are a few other ways to find nodes and relationships based on certain search criteria.  For example, say we wanted to find all instances of "Footloose", movies or otherwise, that were filmed before 2005.  In this case, we can use a conditional query to do this.

openCypher supports the folowing conditionals:

- `>` - greater than
- `>=` - greater than or equal to
- `<` - less than
- `<=` - less than or equal to
- `=` - equal to
- `<>` - not equal

There are also set operations:

- `IN` - test if the output is within a given set
- `NOT IN` - opposite of `IN`

To find all versions of "Footloose" prior to 2005 we can use the following:

In [ ]:
%%oc

MATCH (n)
WHERE n.title = 'Footloose' AND n.year < 2005
RETURN n

If you compare this output to our earlier output showing all "Footloose" titles, this only contains the titles with a release year of earlier than 2005.

As another example, we can combine conditional arguments to find all versions of Footloose filmed from 2000 to 2012:

In [ ]:
%%oc

MATCH (n)
WHERE n.title = 'Footloose' AND n.year < 2012 AND n.year >=2000 
RETURN n

TIP:  This would provide similar functionality to Gremlin's `between(x,y)` step.  The `between(x,y)` step is inclusive/exclusive.

### Review

In this notebook we covered the basics of using openCypher to look at nodes, relationships, and their subsequent properties. We also looked at how to begin filtering on property values. We'll begin to build on these simple queries in the following notebooks and use these patterns as starting and ending conditions for traversing our graph.


### End of Section 01 - Move onto the next notebook.